In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from cmdstanpy import CmdStanModel
import arviz as az

import numpy as np
import matplotlib.pyplot as plt

dp_mix_stan = """
data
{
    int<lower=1> I; // number of areal locations
    int<lower=1> T; // number of time steps
    int<lower=1> P; // number of covariates
    int<lower=1> H; // truncation of stick breaking construction dp
    
    vector[I*T]     y; // output values
    matrix[I*T,P+1] X; // covariate matrix
    // syntax: y(i,t) = y[T*(i-1) + t]
    
    matrix[I,I] W_raw;
      
    // beta
    vector[P+1]     mu_0;
    matrix[P+1,P+1] Sigma_0;
    
    // w_1
    vector[I] mu_w_1;
    
    // alpha
    real<lower=0> a_alpha;
    real<lower=0> b_alpha;
    
    // tau^2
    real<lower=0> a_tau2;
    real<lower=0> b_tau2;
    
    // sigma^2
    real<lower=0> a_sigma2;
    real<lower=0> b_sigma2;
    
    // rho
    real<lower=0> alpha_rho;
    real<lower=0> beta_rho;
    
    // xi
    real<lower=0> alpha_xi;
    real<lower=0> beta_xi;
}

transformed data
{
    vector[T] ones_T;
    for (t in 1:T)
        ones_T[t] = 1;
    
    matrix[T,T] eye_T;
    eye_T = diag_matrix(ones_T);
 
    vector[I] ones_I;
    for (i in 1:I)
        ones_I[i] = 1;
    
    matrix[I,I] eye_I;
    eye_I = diag_matrix(ones_I);
        
    matrix[I,I] W;
    W = diag_matrix(W_raw*ones_I) - W_raw;
}

parameters
{
    real<lower=0> alpha;
    real<lower=0> sigma2;
    real<lower=0> tau2;
    real<lower=0> rho;
    
    vector<lower=0,upper=1>[I] xi_vec;
    matrix[T,I]                ws;
    matrix[P+1,H]              betas; 
    
    vector<lower=0,upper=1>[H-1] vs;
}

transformed parameters
{
    simplex[H] omegas; 
    
    vector<lower=0, upper=1>[H-1] cumprod_one_mv;    
    cumprod_one_mv = exp(cumulative_sum(log1m(vs)));
    
    omegas[1] = vs[1];
    omegas[2:(H-1)] = vs[2:(H-1)] .* cumprod_one_mv[1:(H-2)];
    omegas[H] = cumprod_one_mv[H-1];
    
    vector<lower=-1,upper=1>[I] xis;
    xis = 2*xi_vec - 1;
}

model
{
    alpha  ~ gamma(a_alpha,b_alpha);
    sigma2 ~ inv_gamma(a_sigma2,b_sigma2);
    tau2   ~ inv_gamma(a_tau2,b_tau2);
    rho    ~ beta(alpha_rho,beta_rho);
    xi_vec ~ beta(alpha_xi,beta_xi);
    vs     ~ beta(1,alpha);
    
    matrix[I,I] inv_Q;
    inv_Q = inverse_spd(rho*W + (1-rho)*eye_I);
    
    ws[1,1:I] ~ multi_normal(mu_w_1, tau2*inv_Q);
    
    for (t in 2:T)
        ws[t,1:I] ~ multi_normal(ws[t-1,1:I]*diag_matrix(xis), tau2*inv_Q);
    
    for (h in 1:H)
        betas[1:P+1,h] ~ multi_normal(mu_0, Sigma_0);
        
    for (i in 1:I) 
    {
        vector[H] log_probs;
        
        for (h in 1:H) 
            log_probs[h] = log(omegas[h] + multi_normal_lpdf(y[T*(i-1)+1:i*T] | X[T*(i-1)+1:i*T, 1:P+1]*betas[1:P+1,h] + ws[1:T,i], sigma2*eye_T));
        
        target += log_sum_exp(log_probs);
    }
}

generated quantities 
{
    vector[I] s;
    
    for (i in 1:I) 
    {
        vector[H] log_probs;
        
        for (h in 1:H) 
            log_probs[h] = log(omegas[h] + multi_normal_lpdf(y[T*(i-1)+1:i*T] | X[T*(i-1)+1:i*T, 1:P+1]*betas[1:P+1,h] + ws[1:T,i], sigma2*eye_T));
        
        s[i] = categorical_rng(softmax(log_probs));
    }
}
"""

stan_file = "bayesian.stan"

with open(stan_file, "w") as fp:
    fp.write(dp_mix_stan)
    
dp_mix = CmdStanModel(stan_file=stan_file)

17:42:51 - cmdstanpy - INFO - compiling stan file /home/daan/bayesian.stan to exe file /home/daan/bayesian
17:43:16 - cmdstanpy - INFO - compiled model executable: /home/daan/bayesian


In [7]:
I = 10
T = 7
P = 3
y = np.zeros(I*T)
X = np.ones((I*T,P+1))
mu_0 = np.ones(P+1)
mu_w_1 = np.ones(I)
W_raw = np.ones((I,I))
Sigma_0 = np.eye(P+1)


data = {
    "I": I,
    "T": T,
    "P": P,
    "H": 4,
    "y": y,
    "X": X,
    "W_raw": W_raw,
    "mu_0": mu_0,
    "Sigma_0": Sigma_0,
    "mu_w_1": mu_w_1,
    "a_alpha": 3,
    "b_alpha": 2,
    "a_tau2": 3,
    "b_tau2": 2,
    "a_sigma2": 3,
    "b_sigma2": 2,
    "alpha_rho": 1,
    "beta_rho": 1,
    "alpha_xi": 2,
    "beta_xi": 2
}

fit = dp_mix.sample(data)

17:55:37 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

17:55:37 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
17:55:37 - cmdstanpy - ERROR - Chain [2] error: error during processing Operation not permitted
17:55:37 - cmdstanpy - ERROR - Chain [3] error: error during processing Operation not permitted
17:55:37 - cmdstanpy - ERROR - Chain [4] error: error during processing Operation not permitted


17:55:37 - cmdstanpy - INFO - CmdStan done processing.


RuntimeError: Error during sampling:
Exception: beta_lpdf: Random variable is 5.58071, but must be in the interval [0, 1] (in '/home/daan/bayesian.stan', line 97, column 4 to column 38)
	Exception: beta_lpdf: Random variable is 1.04432, but must be in the interval [0, 1] (in '/home/daan/bayesian.stan', line 97, column 4 to column 38)
	Exception: beta_lpdf: Random variable is 3.88712, but must be in the interval [0, 1] (in '/home/daan/bayesian.stan', line 97, column 4 to column 38)
	Exception: beta_lpdf: Random variable is 1.32322, but must be in the interval [0, 1] (in '/home/daan/bayesian.stan', line 97, column 4 to column 38)
	Exception: beta_lpdf: Random variable is 5.46723, but must be in the interval [0, 1] (in '/home/daan/bayesian.stan', line 97, column 4 to column 38)
	Exception: beta_lpdf: Random variable is 1.75307, but must be in the interval [0, 1] (in '/home/daan/bayesian.stan', line 97, column 4 to column 38)
	Exception: beta_lpdf: Random variable is 3.21336, but must be i